In [39]:
import pandas as pd
from google.colab import drive

In [40]:
# Import source
train = pd.read_csv("/content/drive/MyDrive/Kaggle/Titanic - Machine Learning from Disaster/source/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Kaggle/Titanic - Machine Learning from Disaster/source/test.csv")
test_ids = test["PassengerId"]

def clean_data(df: pd.DataFrame):
  """
    - Remove the unused featutes
    - Fill the numerical NaN values with median
  """
  # Clean the unused features
  df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

  # Fill out the NaN values
  numerical_cols = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
  for col in numerical_cols:
    df.fillna({col: df[col].median()}, inplace=True)

  # Fill out object NaN
  cols = ["Sex", "Embarked"]
  for col in cols:
    df[col].fillna("U", inplace=True)
  return df

train = clean_data(train)
test = clean_data(test)


<ipython-input-40-6e98be96ac08>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna("U", inplace=True)
<ipython-input-40-6e98be96ac08>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method(

In [41]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]
for col in cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train["Survived"]
X = train.drop("Survived", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
clf = LogisticRegression().fit(X_train, y_train)

In [44]:
from sklearn.metrics import accuracy_score
prediction = clf.predict(X_test)
accuracy_score(y_test, prediction)

0.8100558659217877

In [45]:
submission_pred = clf.predict(test)

In [46]:
prediction = pd.DataFrame({"PassengerId": test_ids.values,
                                "Survived": submission_pred})

In [47]:
prediction

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [49]:
prediction.to_csv("prediction.csv", index=False)